In [1]:
using Random
using DataFrames
using GLM
using Statistics
using Plots

In [2]:
Random.seed!(42)
n = 20_000

e1 = randn(n)
e2 = randn(n)
e3 = randn(n)
ex = randn(n)
ey = randn(n)

Z1 = e1
Z3 = e3
Z2 = Z3 .+ e2
X = Z1 .+ Z2 .+ ex
Y = X .+ Z1 .+ Z2 .+ Z3 .+ ey

df = DataFrame(Y=Y, X=X, Z1=Z1, Z2=Z2, Z3=Z3)

In [3]:
function run_ols(vars)
    syms = Symbol.(vars)
    fmla = Term(:Y) ~ sum(Term.(syms))
    model = lm(fmla, df)
    coefs = coef(model)
    se = stderror(model)
    names = coefnames(model)
    idx = findfirst(==("X"), names)
    if idx === nothing
        idx = findfirst(==(:X), names)
    end
    β = coefs[idx]
    seX = se[idx]
    z = 2.5758  # IC 99%
    ci_low = β - z * seX
    ci_high = β + z * seX
    return (vars = vars, β = β, se = seX, ci_low = ci_low, ci_high = ci_high)
end

In [4]:
models = [
    ["X"],
    ["X", "Z1"],
    ["X", "Z2"],
    ["X", "Z1", "Z2"],
    ["X", "Z1", "Z2", "Z3"]
]

In [5]:
results = [run_ols(m) for m in models]

tabla = DataFrame(
    Regresion = ["Y~X", "Y~X+Z1", "Y~X+Z2", "Y~X+Z1+Z2", "Y~X+Z1+Z2+Z3"],
    Beta_X = [r.β for r in results],
    SE = [r.se for r in results],
    CI_low = [r.ci_low for r in results],
    CI_high = [r.ci_high for r in results]
)

In [6]:
scatter(
    1:5, tabla.Beta_X,
    yerr = (tabla.Beta_X .- tabla.CI_low, tabla.CI_high .- tabla.Beta_X),
    legend = false,
    xlabel = "",
    ylabel = "Estimador de β_X",
    title = "Estimaciones del efecto de X sobre Y con IC 99%",
    color = :orange,
    markersize = 6
)
hline!([1.0], linestyle = :dash, color = :black)
xticks!(1:5, tabla.Regresion)
savefig("efecto_X_sobre_Y_julia.png")

Los resultados muestran las estimaciones del efecto de X sobre Y bajo diferentes especificaciones de control. En las primeras tres regresiones (Y~X, Y~X+Z1, Y~X+Z2), los coeficientes estimados de X son significativamente superiores al valor verdadero (β = 1), lo que indica sesgo por variables omitidas. Esto ocurre porque al no incluir simultáneamente las variables Z1 y Z2, se mantienen abiertos los caminos de confusión que influyen tanto en X como en Y.

En cambio, las regresiones Y~X+Z1+Z2 y Y~X+Z1+Z2+Z3 arrojan estimaciones cercanas al valor verdadero, mostrando que controlar por Z1 y Z2 bloquea todos los backdoors y permite identificar correctamente el efecto causal de X sobre Y. La inclusión de Z3 no mejora la estimación, ya que esta variable actúa como ancestro de Z2, y su incorporación únicamente incrementa la colinealidad, reduciendo levemente la precisión.